In [2]:
import pandas as pd
import numpy as np
import string 
import re
import seaborn as sns 
import matplotlib.pyplot as plt 
import warnings
warnings.filterwarnings("ignore")
import nltk
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier

In [3]:
train=pd.read_csv('train_E6oV3lV.csv')
test=pd.read_csv('test_tweets_anuFYb8.csv')
submission=pd.read_csv('sample_submission_gfvA5FD.csv')

In [4]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [5]:
test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [6]:
submission.head()

,id,label
0,31963,0
1,31964,0
2,31965,0
3,31966,0
4,31967,0


In [7]:
train.shape,test.shape

((31962, 3), (17197, 2))

In [8]:
comb=train.append(test,ignore_index=True)

In [9]:
comb.head()

,id,label,tweet
0,1,0.0,@user when a father is dysfunctional and is s...
1,2,0.0,@user @user thanks for #lyft credit i can't us...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation


In [10]:
comb['t1']=comb['tweet'].str.replace('@[\w]*','')

In [11]:
comb['t1'] = comb['t1'].str.replace("[^a-zA-Z#]", " ") 

In [12]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [13]:
def remove_punct(text):
    text_nonpunct=''.join([char for char in text if char not in string.punctuation])
    return text_nonpunct

In [14]:
comb['t2']=comb['t1'].apply(lambda x:remove_punct(x))

In [15]:
comb.head()

,id,label,tweet,t1,t2
0,1,0.0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so sel...,when a father is dysfunctional and is so sel...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for #lyft credit i can t use cause th...,thanks for lyft credit i can t use cause the...
2,3,0.0,bihday your majesty,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...,model i love u take with u all the time in u...
4,5,0.0,factsguide: society now #motivation,factsguide society now #motivation,factsguide society now motivation


In [16]:
comb['token']=comb['t2'].apply(lambda x:x.split())

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shubham\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
stopword=nltk.corpus.stopwords.words('english')

In [19]:
def remove_stopwords(tokenized_list):
    text=[word for word in tokenized_list if word not in stopword]
    return text

In [20]:
comb['nonstop']=comb['token'].apply(lambda x: remove_stopwords(x))

In [21]:
from nltk.stem.wordnet import *

In [22]:
wn=WordNetLemmatizer()

In [23]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shubham\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [24]:
comb['lemma']=comb['nonstop'].apply(lambda x: [wn.lemmatize(i) for i in x])

In [25]:
tokenized_tweet=comb['lemma']

In [26]:
from sacremoses import MosesDetokenizer

In [27]:
detokenizer=MosesDetokenizer()

In [28]:
comb['final']=comb['lemma'].apply(lambda x: detokenizer.detokenize(x))

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

In [30]:
count_vect=CountVectorizer(ngram_range=(1,2),max_df=0.90, min_df=2, stop_words='english')

In [31]:
X_count=count_vect.fit_transform(comb['final'])

In [32]:
X_count

<49159x50778 sparse matrix of type '<class 'numpy.int64'>'
	with 427990 stored elements in Compressed Sparse Row format>

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [34]:
tf=TfidfVectorizer(ngram_range=(1,2),max_df=0.90, min_df=2, stop_words='english')

In [35]:
X_tf=tf.fit_transform(comb['final'])

In [36]:
X_tf

<49159x50778 sparse matrix of type '<class 'numpy.float64'>'
	with 427990 stored elements in Compressed Sparse Row format>

In [37]:
train_bow = X_count[:31962,:] 
test_bow = X_count[31962:,:]

In [38]:
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(train_bow, train['label'],random_state=42,test_size=0.3)

In [39]:
lreg = LogisticRegression() 

In [40]:
lreg.fit(xtrain_bow, ytrain) 

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [41]:
prediction = lreg.predict_proba(xvalid_bow)

In [42]:
prediction_int = prediction[:,1] >= 0.3

In [43]:
prediction_int = prediction_int.astype(np.int)

In [44]:
f1_score(yvalid, prediction_int)

0.689075630252101

In [ ]:
rf = RandomForestClassifier(n_estimators=400,max_features='auto', random_state=11).fit(xtrain_bow, ytrain) 
prediction = rf.predict(xvalid_bow) 
f1_score(yvalid, prediction)

In [ ]:
train_tfidf = X_tf[:31962,:] 
test_tfidf = X_tf[31962:,:] 
xtrain_tfidf = train_tfidf[ytrain.index] 
xvalid_tfidf = train_tfidf[yvalid.index]


In [ ]:
lreg.fit(xtrain_tfidf, ytrain) 
prediction = lreg.predict_proba(xvalid_tfidf) 
prediction_int = prediction[:,1] >= 0.3 
prediction_int = prediction_int.astype(np.int) 
f1_score(yvalid, prediction_int)

In [ ]:
rf = RandomForestClassifier(n_estimators=1500,max_features='auto', random_state=11).fit(xtrain_tfidf, ytrain) 
prediction = rf.predict(xvalid_tfidf) 
f1_score(yvalid, prediction)

In [45]:
test_pred = lreg.predict_proba(test_bow) 
prediction_int = prediction[:,1] >= 0.3
test_pred = prediction_int.astype(np.int)

In [48]:
test_pred=pd.DataFrame(test_pred)

In [69]:
test_pred = rf.predict(test_bow) 

In [70]:
#test_pred = rf.predict(test_tfidf) 

In [49]:
test['label'] = test_pred
submission = test[['id','label']] 

In [50]:
submission.to_csv('d:/sub_rf_bow2.csv', index=False)
